In [72]:
import pdfplumber
import json
import re
from elasticsearch import Elasticsearch
from functools import partial


Config ignore table when extract_text

In [73]:
def not_within_bboxes(obj, bboxes):
    """Check if the object is in any of the table's bbox."""

    def obj_in_bbox(_bbox):
        """Define objects in box.

        See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404
        """
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)

    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)


def filter_tables(page: pdfplumber.page.Page) -> pdfplumber.page.Page:
    if page.find_tables() != []:
        bboxes = [table.bbox for table in page.find_tables()]
        bbox_not_within_bboxes = partial(not_within_bboxes, bboxes=bboxes)
        page = page.filter(bbox_not_within_bboxes)

    return page

Create raw data after read from pdf (ignore table)

In [74]:
with pdfplumber.open("s3-userguide.pdf") as pdf:
    # init raw data array
    lines_data = []
    # loop each page with
    for page_num, page in enumerate(pdf.pages, start=1):

        lines = filter_tables(page).extract_text(layout=True).split("\n")
        for line_num, line in enumerate(lines, start=1):
            lines_data.append(
                {
                    "content": line,
                    "title": lines[2],
                    "section_title": lines[-4],
                }
            )
        page_tables = page.extract_tables()
        for table in page_tables:
            table_rows = [row for row in table]
            for row in table_rows:
                for col in row:
                    if col != "":
                        lines_data.append(
                            {
                                "content": col,
                                "title": lines[2],
                                "section_title": lines[-4],
                            }
                        )
                        break

    with open("raw.json", "w", encoding="utf-8") as f:
        json.dump(lines_data, f, ensure_ascii=False, indent=4)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[['', '', ''], ['', 'Note\nFor more information about using the Amazon S3 Express One Zone storage class with\ndirectory buckets, see What is S3 Express One Zone? and Directory buckets.', ''], ['', '', '']]]
[]
[]
[]
[]
[]
[]
[]
[[['', '', ''], ['', 'Note\nYou can access Amazon S3 and its features only in the AWS Regions that are enabled for\nyour account. For more information about enabling a Region to create and manage AWS\nresources, see Managing AWS Regions in the AWS General Reference.', ''], ['', '', '']]]
[[['', '', ''], ['', 'Note\n• Amazon S3 does not support object locking for concurrent writers. If two PUT requests\nare simultaneously made to the same key, the request with the latest timestamp wins. If\nthis is an issue, you must build an object-locking mechanism into your application.\n• Updates are key-based. There is no way to make atomic updates across keys. For\nexample, you cannot make the u

Clean, add rule and group raw data to data which can import to ES

In [81]:
with open("raw.json", "r", encoding="utf-8") as f:
    lines_data = json.load(f)

filtered_lines_data = []
table_of_contents = {}
current_content = ""
for line_data in lines_data:
    if line_data["content"]:
        content = line_data["content"].strip()
    
        if content != "":
            content = re.sub(r"\s+", " ", content)
            if "........." in content:
                content = re.sub(r"\.+", "@@@", content)
                title = content.split("@@@")[0].strip()
                page_num = content.split("@@@")[1].strip()
                table_of_contents[title] = page_num
                continue
            if (
                "API Version 2006-03-01" not in content
                and "Amazon Simple Storage Service" not in content
                and content != ""
            ):
                if current_content:
                    if content[0] == "-" or content[0] == "•" or content[0].isnumeric():
                        current_content += "\n" + content
                    elif content[0].islower():
                        current_content += " " + content
                    elif current_content[-1] != ".":
                        current_content += " " + content
                    elif current_content[-1] == "," or current_content[-1] == ":":
                        current_content += " " + content
                    elif content[0].islower():
                        current_content += " " + content
                    else:
                        new = {}
                        new["section_title"] = ""
                        page = line_data["section_title"].strip().split(" ")[-1]
                        new["title"] = re.sub(r"\s+", " ", line_data["title"].strip())
                        new["content"] = current_content
                        if page in table_of_contents.values():
                            new["section_title"] = list(table_of_contents.keys())[
                                list(table_of_contents.values()).index(page)
                            ]
                        else:
                            new["section_title"] = (
                                line_data["section_title"]
                                .strip()
                                .split("API Version")[0]
                                .strip()
                            )

                        new["page"] = page
                        if (
                            new["section_title"]
                            and "Table of Contents" not in new["content"]
                        ):
                            filtered_lines_data.append(new)
                        current_content = content
                else:
                    current_content = content

with open("output.json", "w", encoding="utf-8") as f:
    json.dump(filtered_lines_data, f, ensure_ascii=False, indent=4)

Put data to ES

In [80]:
es = Elasticsearch([{"host": "localhost", "port": 9200, "scheme": "http"}])

with open("output.json", "r") as file:
    data = json.load(file)

for doc in data:
    es.index(index="s3_guide", body=doc)

/var/folders/yz/tvpwz_9d6c56fms92tltys800000gn/T/ipykernel_76373/1241684842.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index="s3_guide", body=doc)
